# Graph Traversal Experiments
Adam Haile - 7/26/2024

The goal of these experiments are to refine the current graph system utilized by LLMFlow. This notebook contains a group of multiple experiments including graph DFS denial, and graph implementation systems which would prevent loops from occurring.

## Experiment 1: DFS Loop Detection

Create a graph which automatically detects and denies nodes from being added which would create a cycle. This method only works for directed graphs.

In [16]:
class Graph:
    def __init__(self):
        self.graph = {}

    def add_node(self, node):
        if node not in self.graph:
            self.graph[node] = []

    def add_edge(self, u, v):
        self.add_node(u)
        self.add_node(v)
        self.graph[u].append(v)
        
        if self._has_cycle():
            self.graph[u].remove(v)  # Remove edge if it creates a cycle
            print(f"Adding edge {u} -> {v} creates a cycle. Edge not added.")
        else:
            print(f"Edge {u} -> {v} added successfully.")

    def _has_cycle(self):
        visited = set()
        recursion_stack = set()

        def dfs(node):
            if node not in visited:
                visited.add(node)
                recursion_stack.add(node)

                for neighbor in self.graph[node]:
                    if neighbor not in visited and dfs(neighbor):
                        return True
                    elif neighbor in recursion_stack:
                        return True

                recursion_stack.remove(node)
            return False

        for node in self.graph:
            if node not in visited:
                if dfs(node):
                    return True
        return False

In [17]:
g = Graph()
g.add_node('A') # A
g.add_node('B') # B
g.add_node('C') # C
g.add_node('D') # D
g.add_node('E') # E

In [20]:
g.add_edge('A', 'B') # A -> B
g.add_edge('B', 'C') # A -> B -> C
g.add_edge('C', 'A') # A -> B -> C -> A (cycle)
g.add_edge('C', 'B') # A -> B -> C -> B (cycle)
g.add_edge('B', 'D') # A -> B -> D
g.add_edge('D', 'E') # A -> B -> D -> E
g.add_edge('D', 'C') # A -> B -> D -> C

Edge A -> B added successfully.
Edge B -> C added successfully.
Adding edge C -> A creates a cycle. Edge not added.
Adding edge C -> B creates a cycle. Edge not added.
Edge B -> D added successfully.
Edge D -> E added successfully.
Edge D -> C added successfully.


## Experiment 2: Union Find Loop Detection

Create a graph which automatically detects and denies nodes from being added which would create a cycle in a directed or undirected graph. This method uses a union find data structure to keep track of the connected components of the graph.

In [21]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, u):
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        root_u = self.find(u)
        root_v = self.find(v)
        if root_u != root_v:
            if self.rank[root_u] > self.rank[root_v]:
                self.parent[root_v] = root_u
            elif self.rank[root_u] < self.rank[root_v]:
                self.parent[root_u] = root_v
            else:
                self.parent[root_v] = root_u
                self.rank[root_u] += 1
            return True
        return False

class Graph:
    def __init__(self):
        self.graph = {}
        self.node_index = {}
        self.index = 0
        self.uf = UnionFind(1000)  # Initialize with a large number for simplicity

    def add_node(self, node):
        if node not in self.node_index:
            self.node_index[node] = self.index
            self.index += 1
            self.graph[node] = []

    def add_edge(self, u, v):
        self.add_node(u)
        self.add_node(v)
        if self.uf.union(self.node_index[u], self.node_index[v]):
            self.graph[u].append(v)
            self.graph[v].append(u)
            print(f"Edge {u} <-> {v} added successfully.")
        else:
            print(f"Adding edge {u} <-> {v} creates a cycle. Edge not added.")

In [22]:
# Usage
g = Graph()
g.add_node('A') # A
g.add_node('B') # B
g.add_node('C') # C
g.add_node('D') # D
g.add_node('E') # E

In [23]:
g.add_edge('A', 'B') # A <-> B
g.add_edge('B', 'C') # B <-> C
g.add_edge('C', 'A') # C <-> A (creates a cycle, edge not added)
g.add_edge('C', 'B') # C <-> B (creates a cycle, edge not added)
g.add_edge('B', 'D') # B <-> D
g.add_edge('D', 'E') # D <-> E
g.add_edge('D', 'C') # D <-> C (creates a cycle, edge not added)

Edge A <-> B added successfully.
Edge B <-> C added successfully.
Adding edge C <-> A creates a cycle. Edge not added.
Adding edge C <-> B creates a cycle. Edge not added.
Edge B <-> D added successfully.
Edge D <-> E added successfully.
Adding edge D <-> C creates a cycle. Edge not added.
